## Estratégia Mista (Exemplos)

In [3]:
using SymPy

# produto cartesiano
function matriz_S(x) #função para gerar matriz de estratégias
    S = vec(collect(Base.product(reverse(x)...))) #produto cartesiano
    #transformar conjunto de estratégias em matriz
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end

matriz_S (generic function with 1 method)

In [4]:
#matriz de estratégias
a1 = [Sym("a_$i^$j") for i=1 for j=["c","nc"]]
a2 = [Sym("a_$i^$j") for i=2 for j=["c","nc"]]
A = [a1 , a2]
S = matriz_S(A) 

#matriz de Probabilidades
p1 = [Sym("alpha_$i^$j") for i=1 for j=["c","nc"]]
p2 = [Sym("alpha_$i^$j") for i=2 for j=["c","nc"]]
P = [p1 , p2]
P = matriz_S(P) 

#matriz utilidade
x = symbols("x", integer=true, positive = true, nonzero = true)
U = [2*x -x; -x 2*x;-x 2*x;2*x -x] 



4×2 Array{Sym,2}:
 2⋅x   -x
  -x  2⋅x
  -x  2⋅x
 2⋅x   -x

In [5]:
# a soma das probabilidades para cada jogador deve ser iguala um
function sum_prob(x)
    eqn = []
    for i = 1:size(x, 2) 
        f = Eq(sum(unique(x[:,i])),1)
        if i == 1 eqn = [f; ] else eqn = [eqn f] end
    end
    return eqn
end
eqn1 = sum_prob(P)

1×2 Array{Sym,2}:
 α_1__c + α_1__nc = 1  α_2__c + α_2__nc = 1

In [6]:
#Produto Hadamard (produto entre elementos de uma mesma linha)
#cada linha representa uma probabilidade condicional
function hadamard_product(x)
    ncol = size(x, 2)
    d = []
    for i = 1:(ncol - 1)
        if i == 1 d = (x[:,i] .* x[:,(i+1)]) else d = (x[:,i+1] .* d) end   
    end    
    return d
end

S_P = hadamard_product(P)

#utilidade esperada de cada jogador
eqn2 = S_P.T * U

1×2 Array{Sym,2}:
 2⋅α_1__c⋅α_2__c⋅x - α_1__c⋅α_2__nc⋅x - α_1__nc⋅α_2__c⋅x + 2⋅α_1__nc⋅α_2__nc⋅x  …  -α_1__c⋅α_2__c⋅x + 2⋅α_1__c⋅α_2__nc⋅x + 2⋅α_1__nc⋅α_2__c⋅x - α_1__nc⋅α_2__nc⋅x

In [8]:
#encontrar condições de primeira ordem para cada jogador (pl)
function cpo(eqn1,eqn2,P)
    ee = []
    eqn3 = copy(eqn2)
    for pl in 1:size(P,2) # pl = jogador
        var = unique(P[:,pl])
        for i in 1:(size(var)[1]-1)
            for j in var 
                if j != i 
                    ee = eqn2[pl].subs.(j,solve(eqn1[pl],j)) 
                end
            end
            eq = Eq(diff.(ee[1],var[i]),0)
            eqn3[i,pl] = eq
        end
    end
    return eqn3
end


eqn3 = cpo(eqn1,eqn2,P)

1×2 Array{Sym,2}:
 3⋅α_2__c⋅x - 3⋅α_2__nc⋅x = 0  …  -3⋅α_1__c⋅x + 3⋅α_1__nc⋅x = 0

In [10]:
eqns = vec([eqn3 eqn1])

4-element Array{Sym,1}:
  3⋅α_2__c⋅x - 3⋅α_2__nc⋅x = 0
 -3⋅α_1__c⋅x + 3⋅α_1__nc⋅x = 0
          α_1__c + α_1__nc = 1
          α_2__c + α_2__nc = 1

In [11]:
# variáveis do sistema
#vars = []; for i in eqns.free_symbols append!(vars,i) end; var = reverse(vars) #criar lista de variáveis
vars = unique(P)

solucao = solve(eqns,vars)
sol = [Eq(var[1],var[2]) for var in solucao] 

4-element Array{Sym,1}:
 α_2__nc = 1/2
  α_1__c = 1/2
 α_1__nc = 1/2
  α_2__c = 1/2